In [1]:
# http://annamarbut.blogspot.com/2018/08/whoosh-pandas-and-redshift-implementing.html

In [2]:
import pandas as pd
import glob
import os, os.path

from whoosh.fields import Schema, TEXT
from whoosh import index
from whoosh import index
from whoosh import qparser

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

### Data import

In [4]:
%%time
Run = "../workproduct-files/batchRuns/"
Run_files = glob.glob(Run + "*.pkl")
RunDF = pd.DataFrame()
for filename in Run_files:
    RunDF = RunDF.append(pd.read_pickle(filename))
RunDF = RunDF.sort_index()

Wall time: 4min 6s


In [5]:
%%time
t_data = pd.read_pickle("../workproduct-files/t_dataMaster-keywordsIdentified.pkl")

Wall time: 829 ms


In [6]:
#Combine dataframes leaving out searchTerms from RunDF (duplicate in result)
t_dataRun = pd.concat([t_data, RunDF.iloc[:,1:]], axis = 1, join = "inner")

In [7]:
DF_for_index = t_dataRun

In [8]:
%%time
DF_for_index["wikiMetaPath_str"] = DF_for_index["categoryPath"].apply(lambda x: str(list(x.iloc[:,0])) if type(x) != type(None) else "")
DF_for_index["wikiMetaParents_str"] = DF_for_index["parentCategories"].apply(lambda x: str(list(x.iloc[:,0])) if type(x) != type(None) else "")

Wall time: 5.71 s


### Creating search index

In [9]:
schema = Schema(question = TEXT (stored = True,  field_boost = 2.0), wikiMetaPath = TEXT, wikiMetaParents = TEXT)

In [10]:
# create and populate index
def populate_index(dirname, dataframe, schema):
    
    # Checks for existing index path and creates one if not present
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    print("Creating the Index")
    ix = index.create_in(dirname, schema)
    
    # Imports data from pandas df
    with ix.writer() as writer:
        print("Populating the Index")
        for i in dataframe.index:
            add_data(i, dataframe, writer)

In [11]:
def add_data(i, dataframe, writer):
    
    # writer.update_document(question = str(dataframe.loc[i, "CONS_question"]), wikiMetaPath = str(list(t_dataRun.loc[i, "categoryPath"].iloc[:,0])), wikiMetaParents = str(list(t_dataRun.loc[i, "parentCategories"].iloc[:,0])))
    writer.update_document(question = str(dataframe.loc[i, "CONS_question"]), wikiMetaPath = str(dataframe.loc[i, "wikiMetaPath_str"]), wikiMetaParents = str(dataframe.loc[i, "wikiMetaParents_str"]))

In [12]:
%%time
# Create the index
populate_index("tData_Index1", DF_for_index, schema)

Creating the Index
Populating the Index
Wall time: 46.4 s


### Creating index searcher

In [13]:
def index_search(dirname, search_fields, search_query):
    ix = index.open_dir(dirname)
    schema = ix.schema
    out = []
    
    # Create query parser that looks through designated fields in index
    og = qparser.OrGroup.factory(0.9)
    mp = qparser.MultifieldParser(search_fields, schema, group = og)
    
    # This is the user query
    q = mp.parse(search_query)
    
    # Actual searcher, prints top 10 hits
    with ix.searcher() as s:
        results = s.search(q, limit = 100)
        #print("Search Results: ")
        for row in results:
            #print(row)
            out.append(row["question"])
    return out

### Index search

In [51]:
%%time
a = index_search("tData_Index1", ['question', 'wikiMetaPath', 'wikiMetaParents'], "read book")

Wall time: 46.9 ms


In [53]:
print(len(a))
for row in a:
    print(row)

100
Author of 'Coming of Age in Samoa', the mostly widely read book in the field of anthropology
Which Very Famous Book Which I'm Sure Many Of You Have Read Was Written & Illustrated By “ Eric Carle ” In 1969?
What does a phrenologist read?
What newspaper do the Flintstones read?
Who read the original writing on the wall?
66% of Americans reading on the toilet read what?
What books does me jane read?
Who said 'there, i guess king george can read that'?
On what LP Cover can we read the words 'Welcome Rolling Stones'?
A Maryland t-shirt slogan that parodied 'virginia is for lovers' read what?
Mary Read and Anne Boney had what job in common?
Which of these does Charlie NOT read in The Perks of Being a Wallflower?
Mary Read and Anne Bonny had what job in common?
Team a maryland t-shirt slogan that parodied 'virginia is for lovers' read what?
In the Lego Island trilogy, who taught Pepper Roni how to read?
Credit card on which magnetically encoded information is stored to be read by an elect

In [48]:
%%time
b = index_search("tData_Index1", ['wikiMetaPath', 'wikiMetaParents'], "*second most*")
print(len(b))
for row in b:
    print(row)

11
Who was the only player to win mvp in both leagues?
Vientiane is the capital of ______
Who commanded Bill Jukes Cecco Noodler Skylights Starkey?
What's the oed?
In What Year Were Stock, Aitken & Waterman Taking A Ferry Crosss The Mersey?
How many tunnels under the mersey link liverpool to the wirral?
Hoy and Rousay are part of which British island group?
In which county does the River Mersey rise?
The mersey river runs through _____
Jerry Lee Lewis had Great _____ Of Fire.
In the 1980s, a service called Gameline allowed users to download games to what console?
Wall time: 586 ms


### Testing

In [17]:
type(None)

NoneType

In [18]:
DF_for_index.shape

(50055, 32)

In [19]:
t_dataRun.columns

Index(['CONS_id', 'CONS_question', 'CONS_answer', 'CONS_alt answers',
       'CONS_category', 'CONS_alt categories - NOT USED',
       'CONS_type-formulation', 'CONS_type-multipleChoice', 'ORIG_id',
       'ORIG_question', 'ORIG_answer', 'ORIG_alt answers', 'ORIG_category',
       'ORIG_alt categories', 'ORIG_difficulty', 'ORIG_type', 'Source',
       'Duplicate_removed', 'namedEntities', 'nouns', 'objects', 'subjects',
       'nounsObjectsSubjects', 'searchTerms', 'wikipediaSearchSuccessful',
       'findQuestionCategories_meta', 'wikipediaArticleTitle',
       'wikipediaArticleID', 'categoryPath', 'parentCategories',
       'wikiMetaPath_str', 'wikiMetaParents_str'],
      dtype='object')

In [20]:
t_dataRun.loc[2, "parentCategories"].iloc[:,0]

0                   Dutch-language_books
1     Personal_accounts_of_the_Holocaust
2        CS1_Dutch-language_sources_(nl)
3                  Forgery_controversies
4     CS1_Japanese-language_sources_(ja)
5                   World_War_II_memoirs
6                    Public_domain_books
7           Memory_of_the_World_Register
8           Books_published_posthumously
9                             Anne_Frank
10                     Jewish_literature
11          Books_relating_to_Anne_Frank
12                      Dutch_literature
13              Books_adapted_into_films
14                               Diaries
15           Doubleday_(publisher)_books
Name: pages.title, dtype: object

In [21]:
str(list(t_dataRun.loc[4, "categoryPath"].iloc[:,0]))

"['Peace', 'Nonviolence', 'Violence', 'Society', 'Main_topic_classifications']"